## Import BeautifulSoup and other required libraries
## Create headers for parsing

In [2]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

# Use beautifulsoup to parse the html and store in dataframe by tags

In [5]:
html_doc = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(html_doc, headers)
soup = BeautifulSoup(req.content,'lxml' )
tables = soup.find('table', class_ = 'sortable')
df_cols = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=df_cols)
# Search through table rows, append to list and then push to dataframe
for tr in tables.find_all('tr'):
    rows=[]
    for td in tr.find_all('td'):
        rows.append(td.text.strip())
        if len(rows)==3:
            df.loc[len(df)] = rows


In [7]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Clean up the data

In [8]:
df = df[~((df['Borough'] == 'Not assigned'))]

In [9]:
df['Neighborhood'].loc[df['Neighborhood'] == 'Not assigned'] =  df.Borough

C:\Users\kanda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
df.shape

(103, 3)